<a href="https://colab.research.google.com/github/bvchand/Algorithmic_ToolBox/blob/main/Covid_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [172]:
# Comment this out as I am trying the mount my google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [173]:
# Merging the Stock Indexes and the currency data with rows as dates and columns as index values
# Preprocessing the data sets with online steps for data processing 
# Figuring a way to convert the unemployment data into county wise data - probably using a probability distribution
# Deal with missing stock data for weekends and holidays

Stock Data: Pre-processing


In [174]:
# Reading CSVs for three stock indices
df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/DJI.csv")
df_DJI = df[["Date", "Adj Close", "Volume"]]

df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/NASDAQ.csv")
df_NASDAQ = df[["Date", "Adj Close", "Volume"]]

df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/S&P500.csv")
df_SP500 = df[["Date", "Adj Close", "Volume"]]

In [175]:
# Rename columns in each index in order to merge the indices 
df_NASDAQ = df_NASDAQ.rename(columns={'Adj Close': 'Adj Close NASDAQ', 'Volume': 'Volume NASDAQ'})
df_SP500 = df_SP500.rename(columns={'Adj Close': 'Adj Close SP500', 'Volume': 'Volume SP500'})
df_DJI = df_DJI.rename(columns={'Adj Close': 'Adj Close DJI', 'Volume': 'Volume DJI'})

In [176]:
# Merging data of three stock indices based on column: 'Date' (merging two DFs at once)
df_Merged = pd.merge(df_SP500, df_DJI, on=['Date'])
df_stock_data = pd.merge(df_Merged, df_NASDAQ, on=['Date'])

In [177]:
df_stock_data.head()

,Date,Adj Close SP500,Volume SP500,Adj Close DJI,Volume DJI,Adj Close NASDAQ,Volume NASDAQ
0,2015-10-26,2071.179932,3385800000,17623.050781,116660000,5034.700195,1758690000
1,2015-10-27,2065.889893,4216880000,17581.429688,113960000,5030.149902,1986840000
2,2015-10-28,2090.350098,4698110000,17779.519531,115630000,5095.689941,2141130000
3,2015-10-29,2089.409912,4008940000,17755.800781,90300000,5074.270020,1928310000
4,2015-10-30,2079.360107,4256200000,17663.539063,149250000,5053.750000,2016390000


# COVID-19 Data Loading and Preprocessing

In [182]:
# Function to load the covid_clean_complete dataset from the drive
def load_dataset_us_cases():  
  df_covid_clean_complete = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/COVID_19/covid_19_clean_complete.csv")
  return df_covid_clean_complete

# Function to preprocess the clean_complete_dataset
def preprocess_us_cases_df():
  df_US_daily_cases = load_dataset_us_cases()

  # get the daily data only for US
  df_US_daily_cases = df_covid_clean_complete.loc[df_covid_clean_complete['Country/Region'] == "US"]
  
  df_US_daily_cases = df_US_daily_cases.reset_index(drop=True)

  # Added a column for new_cases daily by getting the differerence of the confirmed cases
  df_US_daily_cases["New_cases"] = df_US_daily_cases["Confirmed"].diff()
  
  df_US_daily_cases['New_cases'] = df_US_daily_cases['New_cases'].fillna(0)

  df_US_daily_cases = df_US_daily_cases.rename(columns={"Confirmed": "Total_Confirmed", "Deaths" : "Total_Deaths"})

  return df_US_daily_cases


# Standardize the COVID variables like New_cases, Total_cases, Total_deaths

In [183]:
# Function to standardize the covid_variables like New_cases, Total_cases, Total_deaths
# Adds new columns to the df with the standarized values
# params - the dataframe, and the column name (COVID Variable) which has to be standarized
def covid_variable_impact_cal(dataframe_to_standardize, column_name):
  col_mean = dataframe_to_standardize[column_name].mean()
  col_std  = dataframe_to_standardize[column_name].std()

  col_name = "Standardized_Value_" + column_name
  dataframe_to_standardize[col_name] = dataframe_to_standardize[column_name]

  i = 0
  j = 0
  while i < len(dataframe_to_standardize):
    j = i
    if i > 7:
      curr_daily_value = dataframe_to_standardize[column_name][i]

      formula_cal_value = (curr_daily_value - col_mean) / col_std

      dataframe_to_standardize.loc[i, col_name] = formula_cal_value

    i += 1

  return dataframe_to_standardize

In [184]:
df_US_daily_cases = preprocess_us_cases_df()
df_US_daily_cases.head()


NameError: ignored

In [185]:
df_standarized = covid_variable_impact_cal(df_US_daily_cases, "Total_Confirmed")

df_standarized.head()

NameError: ignored

## Computing Stock paramaters - Abnormal price and Volatality:

In [162]:
def stock_params(df_stock_data):
  # splitting stock data as pre-covid and covid
  mask = (df_stock_data['Date'] >= '2015-10-26') & (df_stock_data['Date'] <= '2020-01-21')
  historical_stock_data = df_stock_data.loc[mask]

  mask = (df_stock_data['Date'] > '2020-01-21') & (df_stock_data['Date'] <= '2020-07-27') #135 days excluding weekends - 188 days including
  covid_stock_data =  df_stock_data.loc[mask]
  covid_closing_prices = covid_stock_data.iloc[:, lambda df: [1,3,5]]
  
  print("P_t")
  print(covid_closing_prices)

  # compute the average price between 2015-10-26 and 2020-01-21
  history_closing_prices = historical_stock_data.iloc[:, lambda df: [1,3,5]]
  history_total = history_closing_prices.sum()
  avg_history_total = history_total.div(1066)

  print("P_t-i")
  print(avg_history_total)

  # compute standard deviation
  diff = history_closing_prices - [avg_history_total[0], avg_history_total[1], avg_history_total[2]]
  sd_stock_indices = (diff**2 / 1066)**0.5

  print("sigma_P")
  print(sd_stock_indices)


  return historical_stock_data,covid_stock_data


In [163]:
historical_stock_data,covid_stock_data = stock_params(df_stock_data)
# print(historical_stock_data)
# print(covid_stock_data)

P_t
      Adj Close SP500  Adj Close DJI  Adj Close NASDAQ
1066      3321.750000   29186.269531       9383.769531
1067      3325.540039   29160.089844       9402.480469
1068      3295.469971   28989.730469       9314.910156
1069      3243.629883   28535.800781       9139.309570
1070      3276.239990   28722.849609       9269.679688
...               ...            ...               ...
1191      3257.300049   26840.400391      10680.360352
1192      3276.020020   27005.839844      10706.129883
1193      3235.659912   26652.330078      10461.419922
1194      3215.629883   26469.890625      10363.179688
1195      3239.409912   26584.769531      10536.269531

[130 rows x 3 columns]
P_t-i
Adj Close SP500      2538.303169
Adj Close DJI       22624.756401
Adj Close NASDAQ     6608.168708
dtype: float64
sigma_P
      Adj Close SP500  Adj Close DJI  Adj Close NASDAQ
0           14.307141     153.193213         48.192500
1           14.469166     154.467992         48.331867
2           13.7199